A little bit of setup

In [1]:
import sys, os
import pandas as pd
import json
import shutil
import genomitory

## Insert SAMID and experimnet name from HITSLIMS and known sublibrary info
- Please copy fastq path file and library list into the data folder

In [2]:
SAM_ID = 'SAM24449114'
experiment_name="NGS5704"
sublibrary = "1" # choose among 1,2,3,4 or None, If it is not any assigned sublibraries, you have to give the cripsr index file in the data folder

# section (I): Load info

In [5]:
#Create smaple sheet from librray list comes from HITSLIMS

In [ ]:
F = pd.read_excel(f'./data/libraryList_{experiment_name}.xlsx')

In [ ]:
F = F[['LIBID','SAMID','LIBRARY_TYPE']]
F=F.rename(columns={'LIBRARY_TYPE':'LIBRARY'})
F.index = F.index.values
F['index'] =F.index
max_run=len(F)//3
F['SAMPLE'] = F['index'].apply(lambda x:x % max_run+1) 
F = F[['LIBID', 'SAMID', 'LIBRARY', 'SAMPLE']]
F['LIBRARY'] = F['LIBRARY'].replace({'Expression':'GEX'})
F.to_csv("./data/RECRUSION_DLD1_DAY5_SAMPLE_SHEET.tsv",sep='\t', index=False)

In [ ]:
#this comes from hitsLims
fastq = pd.read_excel(f'./data/{experiment_name}_ProjectFastq.xlsx').rename(columns={'SAMID':'Pool SAMID', 'Data Path':'Flowcell'})
fastq = fastq[fastq['FASTQ QC Status'].isin(['TESTRUN', 'OK'])]
#this comes from Russell
exp = pd.read_csv('./data/RECRUSION_DLD1_DAY5_SAMPLE_SHEET.tsv', sep='\t').rename(columns={'SAMID':'Pool SAMID','LIBRARY':'Library type',
                                                                                                  'SAMPLE': 'replicate'})

In [ ]:
exp

In [ ]:
fastq

## Calculate the size of fastq files

In [ ]:
fastq_paths = fastq["Flowcell"].unique().tolist()

In [ ]:
def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            total_size += os.path.getsize(file_path)
    return total_size


In [ ]:
combined_size_bytes = sum(get_folder_size(fastq_path) for fastq_path in fastq_paths)

# Convert bytes to GB
combined_size_gb = combined_size_bytes / (1024 ** 3)

# Print the combined size in GB
print(f"Combined size of folders: {combined_size_gb:.2f} GB")
print(f"Number of 10x runs: {max_run}")

## Decide the number of splits based on data

In [ ]:
N_splits = 4

# Section (II): Make the required directories

In [3]:
cur = os.getcwd()

In [4]:
cellR, demuxH, demuxc = {},{},{}
for i in range(N_splits):
    folder = os.path.abspath(os.path.join(cur ,"Split_"+str(i+1)))
    try:
        os.mkdir(folder)
    except FileExistsError:
        pass
    cellR[i] = os.path.abspath(os.path.join(folder ,"cellranger"))
    try:    
        os.mkdir(cellR[i])
    except FileExistsError:
        pass
    demuxH[i] = os.path.abspath(os.path.join(folder ,"demuxem_hashing"))
    try:
        os.mkdir(demuxH[i])
    except FileExistsError:
        pass
    demuxc[i] = os.path.abspath(os.path.join(folder ,"demuxem_crispr"))
    try:
        os.mkdir(demuxc[i])
    except FileExistsError:
        pass

In [5]:
reference = 'gs://gred-melo-carlos-lab/genomes/GRCh38_TurboGFP_3xLinker/GRCh38-2020-A_and_TurboGFP_and_3xLinker.tar.gz'

library_map = {"GEX":'rna',
 'HTO':'hashing',
 'gRNA':'crispr'
}

In [15]:
fastq = fastq[fastq['Pool SAMID']==SAM_ID].reset_index().copy()

In [16]:
fastq

,index,PROJECT ID,RUNID,Pool SAMID,LIBID,Flowcell,Read1,Read2,Read3,FASTQ QC Status,Library Subtype,Read Count
0,0,NGS5704,R10193,SAM24449114,LIB6229736,/gne/data/dnaseq/processed_runs/R10193/results,LIB6229736_SAM24449114_S25_L005_R1_001.fastq.gz,LIB6229736_SAM24449114_S25_L005_R2_001.fastq.gz,NaN,OK,Expression,452582214
1,1,NGS5704,R10193,SAM24449114,LIB6229736,/gne/data/dnaseq/processed_runs/R10193/results,LIB6229736_SAM24449114_S25_L006_R1_001.fastq.gz,LIB6229736_SAM24449114_S25_L006_R2_001.fastq.gz,NaN,OK,Expression,456593733
2,2,NGS5704,R10193,SAM24449114,LIB6229737,/gne/data/dnaseq/processed_runs/R10193/results,LIB6229737_SAM24449114_S26_L005_R1_001.fastq.gz,LIB6229737_SAM24449114_S26_L005_R2_001.fastq.gz,NaN,OK,Expression,476188806
3,3,NGS5704,R10193,SAM24449114,LIB6229737,/gne/data/dnaseq/processed_runs/R10193/results,LIB6229737_SAM24449114_S26_L006_R1_001.fastq.gz,LIB6229737_SAM24449114_S26_L006_R2_001.fastq.gz,NaN,OK,Expression,480781247
4,4,NGS5704,R10193,SAM24449114,LIB6229738,/gne/data/dnaseq/processed_runs/R10193/results,LIB6229738_SAM24449114_S27_L005_R1_001.fastq.gz,LIB6229738_SAM24449114_S27_L005_R2_001.fastq.gz,NaN,OK,Expression,424573848
...,...,...,...,...,...,...,...,...,...,...,...,...
77,77,NGS5704,R10193,SAM24449114,LIB6229723,/gne/data/dnaseq/processed_runs/R10193/results,LIB6229723_SAM24449114_S32_L008_R1_001.fastq.gz,LIB6229723_SAM24449114_S32_L008_R2_001.fastq.gz,NaN,OK,gRNA,21400521
78,78,NGS5704,R10200,SAM24449114,LIB6229723,/gne/data/dnaseq/processed_runs/R10200/results,LIB6229723_SAM24449114_S27_L004_R1_001.fastq.gz,LIB6229723_SAM24449114_S27_L004_R2_001.fastq.gz,NaN,OK,gRNA,15230018
79,79,NGS5704,R10193,SAM24449114,LIB6229724,/gne/data/dnaseq/processed_runs/R10193/results,LIB6229724_SAM24449114_S33_L007_R1_001.fastq.gz,LIB6229724_SAM24449114_S33_L007_R2_001.fastq.gz,NaN,OK,gRNA,17957975
80,80,NGS5704,R10193,SAM24449114,LIB6229724,/gne/data/dnaseq/processed_runs/R10193/results,LIB6229724_SAM24449114_S33_L008_R1_001.fastq.gz,LIB6229724_SAM24449114_S33_L008_R2_001.fastq.gz,NaN,OK,gRNA,16520657


In [17]:
data = pd.merge(
    fastq[['Pool SAMID', 'LIBID', 'Flowcell', 'Read1']],
    exp[['Pool SAMID', 'LIBID', 'Library type', 'replicate']]
)
data['Library'] = data['Read1'].apply(lambda x:x[:22])
data['Sample'] = data['Pool SAMID'] + '_rep' + data['replicate'].astype(str)
data['Library type'] = data['Library type'].map(library_map)
data = data.rename(columns={'Library type':'DataType'})
data = data[['Library', 'Sample', 'Flowcell', 'DataType']].drop_duplicates().sort_values(by=['Sample', 'DataType'], ascending=[True, False]).reset_index(drop=True)

In [18]:
data

,Library,Sample,Flowcell,DataType
0,LIB6229736_SAM24449114,SAM24449114_rep1,/gne/data/dnaseq/processed_runs/R10193/results,rna
1,LIB6229701_SAM24449114,SAM24449114_rep1,/gne/data/dnaseq/processed_runs/R10193/results,hashing
2,LIB6229713_SAM24449114,SAM24449114_rep1,/gne/data/dnaseq/processed_runs/R10193/results,crispr
3,LIB6229745_SAM24449114,SAM24449114_rep10,/gne/data/dnaseq/processed_runs/R10193/results,rna
4,LIB6229710_SAM24449114,SAM24449114_rep10,/gne/data/dnaseq/processed_runs/R10193/results,hashing
5,LIB6229722_SAM24449114,SAM24449114_rep10,/gne/data/dnaseq/processed_runs/R10193/results,crispr
6,LIB6229722_SAM24449114,SAM24449114_rep10,/gne/data/dnaseq/processed_runs/R10200/results,crispr
7,LIB6229746_SAM24449114,SAM24449114_rep11,/gne/data/dnaseq/processed_runs/R10193/results,rna
8,LIB6229711_SAM24449114,SAM24449114_rep11,/gne/data/dnaseq/processed_runs/R10193/results,hashing
9,LIB6229723_SAM24449114,SAM24449114_rep11,/gne/data/dnaseq/processed_runs/R10193/results,crispr


In [19]:
cellR

{0: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_1/cellranger',
 1: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_2/cellranger',
 2: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_3/cellranger',
 3: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_4/cellranger'}

In [20]:
demuxH

{0: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_1/demuxem_hashing',
 1: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_2/demuxem_hashing',
 2: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_3/demuxem_hashing',
 3: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_4/demuxem_hashing'}

In [21]:
demuxc

{0: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_1/demuxem_crispr',
 1: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_2/demuxem_crispr',
 2: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_3/demuxem_crispr',
 3: '/gstore/home/ghaffars/Cumulus/crc_ngs5704/Cumulus/Split_4/demuxem_crispr'}

In [22]:
data.Sample.unique().tolist()

['SAM24449114_rep1',
 'SAM24449114_rep10',
 'SAM24449114_rep11',
 'SAM24449114_rep12',
 'SAM24449114_rep2',
 'SAM24449114_rep3',
 'SAM24449114_rep4',
 'SAM24449114_rep5',
 'SAM24449114_rep6',
 'SAM24449114_rep7',
 'SAM24449114_rep8',
 'SAM24449114_rep9']

# Section III: Creating required data in the folders

## Split dataset into a number of splits to create some sublists

### Define a function for that reason

In [23]:
def divide_list (List, N_splits):
    divison_size = len(List) // N_splits
    remainder = len(List) % N_splits
    
    if remainder != 0:
      raise Exception("Sorry, list should be divided to a number of splits")
    
    sorted_list = sorted(List, key=lambda x:int(x[15:]))
    final_list = [element.split('_')[1] for element in sorted_list]
    
    
    divided_list=[]
    start_idx = 0
    for i in range(N_splits):
        division = final_list[start_idx:start_idx+divison_size]
        divided_list.append(division)
        start_idx += divison_size
        
    return divided_list

In [24]:
loop = divide_list(data.Sample.unique().tolist(),N_splits)

In [25]:
loop

[['rep1', 'rep2', 'rep3'],
 ['rep4', 'rep5', 'rep6'],
 ['rep7', 'rep8', 'rep9'],
 ['rep10', 'rep11', 'rep12']]

## Write SampleSheet

In [31]:
def write_sample_sheet(df, path, suffix, chem='threeprime', ref='gs://gred-melo-carlos-lab/genomes/GRCh38_TurboGFP_3xLinker/GRCh38-2020-A_and_TurboGFP_and_3xLinker.tar.gz'):
    header = "Library" + ",Sample" + ",Reference" + ",Flowcell" + ",Chemistry" + ",DataType" + ",FeatureBarcodeFile\n"
    outfile = os.path.join(path, f'samplesheet_{suffix}.csv')
    with open(outfile, 'w') as outf:
        outf.write(f'{header}')
        for el in range(len(df)):
            library=df['Library'][el]
            sample=df['Sample'][el]
            ref=ref
            chem=chem
            datatype=df['DataType'][el]
            flowcell=df['Flowcell'][el]
            if datatype=='rna':
                outf.write(f"{library},{sample},{ref},{flowcell},{chem},{datatype}\n")
            elif datatype=='hashing':                
                feature='hashing_index.csv'
                outf.write(f"{library},{sample},{ref},{flowcell},{chem},{datatype},{feature}\n")
            elif datatype=='crispr':
                feature='crispr_index.csv'
                outf.write(f"{library},{sample},{ref},{flowcell},{chem},{datatype},{feature}\n")

In [32]:
for i in range(N_splits):
    data_s = data[data["Sample"].apply(lambda x:x.split('_')[1]).isin(loop[i])].reset_index()
    write_sample_sheet(data_s, cellR[i], SAM_ID+"_"+str(i+1))

In [33]:
"""
for i in range(N_splits):
    file = os.path.join(cellR[i], f'samplesheet_SAM24432714_{i+1}.csv')
    os.remove(file)
"""

"\nfor i in range(N_splits):\n    file = os.path.join(cellR[i], f'samplesheet_SAM24432714_{i+1}.csv')\n    os.remove(file)\n"

## Write inputs

In [34]:
for i in range(N_splits):
    s2 = SAM_ID
    s = SAM_ID+"_"+str(i+1)
    inputs_cr = {
    "cellranger_workflow.input_csv_file": f"samplesheet_{s}.csv",
    "cellranger_workflow.output_directory": f"gs://gred-melo-carlos-lab/{experiment_name}/{s}/",
    "cellranger_workflow.run_mkfastq": False,
    "cellranger_workflow.expect_cells": 20000,
    "cellranger_workflow.crispr_barcode_pos": 19,
    "cellranger_workflow.max_mismatch": 0,
    "cellranger_workflow.write_to_dsdb": True,
    "cellranger_workflow.dsdb_memory": "120G",
    "cellranger_workflow.dsdb_disk_space":"5000",
    "cellranger_workflow.dsdb_project_authors": "melocars,ghaffars,borozani",
    "cellranger_workflow.dsdb_project_title": f"Recursion DLD1 Library4 Screen-2 (NGS5704)",
    "cellranger_workflow.dsdb_project_description": f"Production scale screen in DLD-1 cells with 1/4 of the genome-wide CRISPR library (sub-lib-4, ~22K guides). 1 time point: Day-5. We performed 12 rxns of 10x 3' HT kit with an estimated loading of 52K cells.We performed a pilot study to test the library quality before the production-level sequencing(NGS5703). We will submit 36 libraries (12 GEX, 12 HTO and 12 sgRNA) We estimate needing - 600M per GEX, - 20M per HTO and - 50M per sgRNA libraries now that we have analyzed the QC run. This run is to add additional cells needed for analysis."
        }
    json_object = json.dumps(inputs_cr, indent=4)
    outfile = os.path.join(cellR[i], f'inputs_{s2}.json')
    with open(outfile, "w") as outfile:
        outfile.write(json_object)

## Prepare submit.sh file

In [37]:
for i in range(N_splits):
    s = SAM_ID+"_"+str(i+1)
    inputs_cr = f"""\
#source /gstore/home/yangy197/miniconda3/bin/activate cumulus
alto cromwell run -s cumulus.gcp.science.roche.com \\
  -m cumulus/cellranger:3.0 \\
  -i inputs_{SAM_ID}.json \\
  -o inputs_updated.json \\
  -b gs://gred-melo-carlos-lab/{experiment_name}/{s}/uploads/ \\
  --no-ssl-verify \\
  --no-cache \\
  --profile dssc-cumulus | tee {experiment_name}.txt
  """
    outfile = os.path.join(cellR[i], f'submit.sh')
    with open(outfile, "w") as outfile:
        outfile.write(inputs_cr)

## After submission, get the IDs and prepare to get metadata

In [26]:
jobids = []

for i in range(N_splits):
    outfile = os.path.join(cellR[i], f'{experiment_name}.txt')
    with open(outfile, 'r') as file:
            for line in file:
                if 'Job ' in line:
                    jobids.append(line.split(' ')[1])

In [27]:
jobids

['8636e67b-9b70-474b-ad43-362cfc01117d',
 '43a564d0-d206-4245-a761-498b8dcc0c68',
 '01d7316c-2e49-4926-87fd-b05e5d669059',
 '376aa376-3ede-4641-b3f5-b52cc26c0611']

## Write a get metadata file

In [29]:
for i in range(N_splits):
    outfile = os.path.join(cellR[i], f'get_metadata.sh')
    with open(outfile, 'w') as f:
        f.write("#source /gstore/project/cumulus/miniconda3/bin/activate cumulus\n")
        f.write(f"alto cromwell get_metadata -s cumulus.gcp.science.roche.com --id {jobids[i]} \n")

## Read DSDB_ids from json metadata files

In [30]:
dsdb = []

for i in range(N_splits):
    infile = os.path.join(cellR[i], f'{jobids[i]}.metadata.json')
    with open(infile, 'r') as f:
        tmp = json.load(f)
        dsdb.append(tmp["outputs"]["cellranger_workflow.dsdb_info"][0]['id'])

In [31]:
dsdb

['DS000016632', 'DS000016634', 'DS000016633', 'DS000016631']

# DemuxEM

## SampleSheet

In [32]:
def create_demuxem_ss(data_s, barcode_datatype, path):
    cols = ['Library', 'Sample', 'DataType']
    df = pd.merge(data_s[data_s['DataType']=='rna'][cols], data_s[data_s['DataType']==barcode_datatype][cols], on='Sample').drop_duplicates()[
        ['Library_x', 'Sample', 'Library_y']]
    df.columns = ['RNA', 'OUTNAME', 'TagFile']
    df['RNA'] = df['RNA'].apply(lambda x:path+x+'/raw_feature_bc_matrix.h5')
    df['TagFile'] = df['TagFile'].apply(lambda x:path+x+'/'+x+f'.{barcode_datatype}.csv')
    df['TYPE'] = 'cell-hashing'
    return df[['OUTNAME', 'RNA', 'TagFile', 'TYPE']]

In [33]:
for i in range(N_splits):
    s2 = SAM_ID+"_"+str(i+1)
    data_s = data[data["Sample"].apply(lambda x:x.split('_')[1]).isin(loop[i])].reset_index()
    for t in ['hashing', 'crispr']:
            demux_s = create_demuxem_ss(data_s, t , f'gs://gred-melo-carlos-lab/{experiment_name}/{s2}/')
            if t=='hashing':
                outfile = os.path.join(demuxH[i], f'demux_samplesheet_{s2}.csv')
            if t=='crispr':
                outfile = os.path.join(demuxc[i], f'demux_samplesheet_{s2}.csv')
            demux_s.to_csv(outfile, index=False)

## jason

In [34]:
for i in range(N_splits):
    s2 = SAM_ID
    s = SAM_ID+"_"+str(i+1)
    for t in ['hashing', 'crispr']:
        if t == 'crispr':
            ab_map = f"{t}_demux_index_map.csv"
            outfile = os.path.join(demuxc[i], f'demux_inputs_{s2}.json')
        if t=='hashing':
            ab_map = f"{t}_demux_index_map.csv"
            outfile = os.path.join(demuxH[i], f'demux_inputs_{s2}.json')
        inputs_cr = {
                "demultiplexing.input_sample_sheet" : f"demux_samplesheet_{s}.csv",
                "demultiplexing.output_directory" : f"gs://gred-melo-carlos-lab/{experiment_name}/{s}/demuxem",
                "demultiplexing.genome" : "gs://gred-melo-carlos-lab/genomes/GRCh38_TurboGFP/GRCh38-2020-A_and_TurboGFP.tar.gz",
                "demultiplexing.demuxEM_memory":"120G",
                "demultiplexing.antibody_index_mapping_csv" : ab_map,
                "demultiplexing.dsdb_id" : dsdb[i],
                "demultiplexing.dsdb_project_authors" : "melocars,ghaffars,borozani",
        }
        json_object = json.dumps(inputs_cr, indent=4)
        with open(outfile, "w") as outfile:
            outfile.write(json_object)        

In [42]:
"""
for i in range(N_splits):
    s = "SAM24432714"
    file = os.path.join(demuxc[i], f'demux_inputs_{s}.json')
    os.remove(file)
"""

'\nfor i in range(N_splits):\n    s = "SAM24432714"\n    file = os.path.join(demuxc[i], f\'demux_inputs_{s}.json\')\n    os.remove(file)\n'

## Prepare submit.sh file

In [ ]:
for i in range(N_splits):
    s = SAM_ID+"_"+str(i+1)
    for t in ['hashing', 'crispr']:
        inputs_cr = f"""\
#source /gstore/home/yangy197/miniconda3/bin/activate cumulus
alto cromwell run -s cumulus.gcp.science.roche.com \\
  -m cumulus/demultiplexing:3.0 \\
  -i demux_inputs_{SAM_ID}.json \\
  -o demux_inputs_updated.json \\
  -b gs://gred-melo-carlos-lab/{experiment_name}/{s}/uploads/ \\
  --profile dssc-cumulus 
  """
        if t=='hashing':
            outfile = os.path.join(demuxH[i], f'submit.sh')
        if t=='crispr':
            outfile = os.path.join(demuxc[i], f'submit.sh')
        with open(outfile, "w") as outfile:
            outfile.write(inputs_cr)

In [ ]:
"""
for i in range(N_splits):
    s2 = "SAM24432714"+"_"+str(i+1)
    s = SAM_ID
    for t in ['hashing', 'crispr']:
        if t=='hashing':
            file = os.path.join(demuxH[i], f'demux_samplesheet_{s}.csv')
        if t=='crispr':
            file = os.path.join(demuxc[i], f'demux_samplesheet_{s}.csv')
        os.remove(file)
"""       

# Section IV : Hashing and Crispr info

## First revise the hashing_map file 

In [35]:
hashing_map = pd.read_csv('./data/hashing.csv')

In [36]:
replacement_dict = {"SAM24434722":"REPLICATE_1","SAM24434723":"REPLICATE_2", "SAM24434748":"REPLICATE_3"}

In [37]:
hashing_map["sample"]=hashing_map["sample"].replace(replacement_dict)

In [38]:
hashing_map

,index,sample,cellline,timepoint,HTO
0,HTO-1,REPLICATE_1,DLD1,day5,HTO-1
1,HTO-2,REPLICATE_1,DLD1,day5,HTO-2
2,HTO-3,REPLICATE_1,DLD1,day5,HTO-3
3,HTO-4,REPLICATE_2,DLD1,day5,HTO-4
4,HTO-5,REPLICATE_2,DLD1,day5,HTO-5
5,HTO-6,REPLICATE_2,DLD1,day5,HTO-6
6,HTO-7,REPLICATE_3,DLD1,day5,HTO-7
7,HTO-8,REPLICATE_3,DLD1,day5,HTO-8
8,HTO-9,REPLICATE_3,DLD1,day5,HTO-9
9,HTO-10,REPLICATE_1,DLD1,day5,HTO-10


In [39]:
hashing_map["NGS_ID"] = experiment_name

In [40]:
hashing_map["Biological_replicate"]=hashing_map["sample"].copy()

In [41]:
hashing_map["sublibrary"] = f"GMTY210:cropseq.crisprko.cas9.human.lib{sublibrary}.concerto@REVISION-3"

In [42]:
hashing_map["gRNA_library_MOI"] = "0.4"

In [43]:
replacement_dict2 = {"REPLICATE_1":"SAM24439043","REPLICATE_2":"SAM24439044", "REPLICATE_3":"SAM24439045"}

In [44]:
hashing_map["sample"]=hashing_map["sample"].replace(replacement_dict2)

In [45]:
hashing_map

,index,sample,cellline,timepoint,HTO,NGS_ID,Biological_replicate,sublibrary,gRNA_library_MOI
0,HTO-1,SAM24439043,DLD1,day5,HTO-1,NGS5704,REPLICATE_1,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4
1,HTO-2,SAM24439043,DLD1,day5,HTO-2,NGS5704,REPLICATE_1,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4
2,HTO-3,SAM24439043,DLD1,day5,HTO-3,NGS5704,REPLICATE_1,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4
3,HTO-4,SAM24439044,DLD1,day5,HTO-4,NGS5704,REPLICATE_2,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4
4,HTO-5,SAM24439044,DLD1,day5,HTO-5,NGS5704,REPLICATE_2,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4
5,HTO-6,SAM24439044,DLD1,day5,HTO-6,NGS5704,REPLICATE_2,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4
6,HTO-7,SAM24439045,DLD1,day5,HTO-7,NGS5704,REPLICATE_3,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4
7,HTO-8,SAM24439045,DLD1,day5,HTO-8,NGS5704,REPLICATE_3,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4
8,HTO-9,SAM24439045,DLD1,day5,HTO-9,NGS5704,REPLICATE_3,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4
9,HTO-10,SAM24439043,DLD1,day5,HTO-10,NGS5704,REPLICATE_1,GMTY210:cropseq.crisprko.cas9.human.lib4.conce...,0.4


In [46]:
hashing_map.to_csv('./data/hashing_demux_index_map.csv', index=False)

## Cripsr info

In [ ]:
gRNA = genomitory.get_features(f"GMTY210:cropseq.crisprko.cas9.human.lib{sublibrary}.concerto@REVISION-3")

In [ ]:
gRNA ['index'] = gRNA ['ID']
gRNA ['sample'] = gRNA ['ID']

In [ ]:
crispr_demux_index = gRNA[['index','sample', 'gene_symbol','class']]

In [ ]:
crispr_demux_index[crispr_demux_index.gene_symbol.isna()]

In [ ]:
crispr_demux_index['gene_symbol'] = np.where(crispr_demux_index['gene_symbol'].isna(), crispr_demux_index['sample'].apply(lambda x:x.split('_')[0]),
                                    crispr_demux_index['gene_symbol'])

In [ ]:
crispr_demux_index[crispr_demux_index.gene_symbol.isna()]

In [ ]:
crispr_demux_index.to_csv('./data/crispr_demux_index_map.csv', index=False)

In [ ]:
crispr_index = gRNA[['index','spacer_19mer']]

In [ ]:
crispr_index["crispr"] = "crispr"

In [ ]:
crispr_index=crispr_index[['spacer_19mer','index','crispr']]

In [ ]:
crispr_index.to_csv('./data/crispr_index.csv', index=False, header=False)

In [ ]:
for i in range(N_splits):
    shutil.copy('./data/crispr_index.csv',cellR[i])
    shutil.copy('./data/hashing_index.csv',cellR[i])

In [7]:
for i in range(N_splits):
    shutil.copy('./data/crispr_demux_index_map.csv',demuxc[i])
    shutil.copy('./data/hashing_demux_index_map.csv',demuxH[i])